In [3]:
# ==============================================================================
# CÉLULA 1: IMPORTAÇÃO DE BIBLIOTECAS E CONFIGURAÇÃO
# ==============================================================================
import pandas as pd
import numpy as np
import io
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
import joblib
import warnings
import kagglehub

warnings.filterwarnings('ignore')

print("Bibliotecas importadas com sucesso.")


Bibliotecas importadas com sucesso.


In [4]:

# ==============================================================================
# CÉLULA 2: CARREGAMENTO E PREPARAÇÃO DOS DADOS (COM AMBOS OS CSVs)
# ==============================================================================

try:
    path = kagglehub.dataset_download("piterfm/massive-missile-attacks-on-ukraine")

    print("Path to dataset files:", path)

    attacks_df = pd.read_csv(f"{path}/missile_attacks_daily.csv")
    weapons_df= pd.read_csv(f"{path}/missiles_and_uav.csv")
except FileNotFoundError:
    print("Erro: Verifique se o arquivo está no diretório correto.")
    exit()



df = pd.merge(attacks_df, weapons_df[['model', 'category']], on='model', how='left')
df['event_date'] = pd.to_datetime(df['time_start'], format='mixed', errors='coerce')
df.dropna(subset=['event_date'], inplace=True)
df['year'] = df['event_date'].dt.year
numeric_cols = ['launched', 'destroyed']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
df['destroyed'] = df.apply(lambda row: min(row['launched'], row['destroyed']), axis=1)
df['category'] = df['category'].fillna('Unknown')
df.rename(columns={'category': 'weapon_category'}, inplace=True)


Path to dataset files: /home/eznolin/.cache/kagglehub/datasets/piterfm/massive-missile-attacks-on-ukraine/versions/136


In [5]:

# ==============================================================================
# CÉLULA 3: CÁLCULO DO P(TRACK) PONDERADO
# ==============================================================================
print("\n--- Calculando P(track) Médio Ponderado com Base no Inventário ---")
inventory_data = {
    'system': ['Patriot PAC-3', 'SAMP/T', 'IRIS-T SLM', 'NASAMS', 'S-300', 'Buk-M1', 'Gepard'],
    'quantity': [3, 1, 13, 15, 100, 60, 52],
    'p_track_tier': [0.99, 0.99, 0.98, 0.98, 0.90, 0.90, 0.85]
}
inventory_df = pd.DataFrame(inventory_data)
total_quantity = inventory_df['quantity'].sum()
weighted_sum = (inventory_df['quantity'] * inventory_df['p_track_tier']).sum()
weighted_p_track = weighted_sum / total_quantity
P_TRACK = round(weighted_p_track, 3)
print(f"O P(track) médio ponderado calculado é: {P_TRACK}")



--- Calculando P(track) Médio Ponderado com Base no Inventário ---
O P(track) médio ponderado calculado é: 0.9


In [6]:

# ==============================================================================
# CÉLULA 4: OBJETIVO 1 - CÁLCULO DA CAPACIDADE DE DEFESA (SSPK IMPLÍCITO)
# ==============================================================================
print("\n--- OBJETIVO 1: Calculando a Capacidade de Defesa (SSPK) ---")
N_SHOTS_MAPPING = {'ballistic missile': 2, 'cruise missile': 2, 'UAV': 1, 'guided bomb': 1, 'surface-to-air missile': 2, 'Unknown': 1}
def calculate_implied_sspk(launched, destroyed, p_track, n_shots):
    if launched == 0: return np.nan
    q = (launched - destroyed) / launched
    kw = 1 - q
    base_term = 1 - (kw / p_track)
    if base_term < 0: return 0.0
    sspk = 1 - (base_term)**(1 / n_shots)
    return sspk

stats_by_year = df.groupby(['year', 'weapon_category']).agg(launched=('launched', 'sum'), destroyed=('destroyed', 'sum')).reset_index()
stats_by_year['n_shots'] = stats_by_year['weapon_category'].str.lower().map(N_SHOTS_MAPPING).fillna(1)
stats_by_year['implied_sspk'] = stats_by_year.apply(lambda row: calculate_implied_sspk(row['launched'], row['destroyed'], P_TRACK, row['n_shots']), axis=1)
sspk_pivot_table = stats_by_year.pivot_table(index='weapon_category', columns='year', values='implied_sspk')
sspk_pivot_table = sspk_pivot_table.applymap(lambda x: f"{x:.1%}" if pd.notnull(x) else "N/A")
print("\nEvolução Anual do SSPK Implícito (Capacidade de Interceptação):")
print(sspk_pivot_table)

stats_by_category = df.groupby('weapon_category').agg(launched=('launched', 'sum'), destroyed=('destroyed', 'sum')).reset_index()
stats_by_category['n_shots'] = stats_by_category['weapon_category'].str.lower().map(N_SHOTS_MAPPING).fillna(1)
stats_by_category['implied_sspk'] = stats_by_category.apply(lambda row: calculate_implied_sspk(row['launched'], row['destroyed'], P_TRACK, row['n_shots']), axis=1)
print("\nSSPK Implícito por Categoria de Arma (Geral):")
print(stats_by_category[stats_by_category['launched'] > 0].sort_values('implied_sspk', ascending=False).round(3))



--- OBJETIVO 1: Calculando a Capacidade de Defesa (SSPK) ---

Evolução Anual do SSPK Implícito (Capacidade de Interceptação):
year                     2022   2023   2024   2025
weapon_category                                   
UAV                      0.0%  94.6%  78.1%  60.6%
Unknown                 92.6%  74.7%  85.6%  15.9%
ballistic missile         N/A  27.2%   7.8%  10.8%
cruise missile          54.7%  58.9%  49.9%  44.1%
guided bomb               N/A  88.9%  27.8%    N/A
surface-to-air missile   0.0%   0.0%   0.5%   0.0%

SSPK Implícito por Categoria de Arma (Geral):
          weapon_category  launched  destroyed  n_shots  implied_sspk
1                 Unknown     398.0      292.0      1.0         0.815
4             guided bomb      14.0        9.0      1.0         0.714
0                     UAV   37355.0    23455.0      1.0         0.698
3          cruise missile    3719.0     2594.0      2.0         0.526
2       ballistic missile     500.0       84.0      2.0         0.09

In [7]:

# ==============================================================================
# CÉLULA 5: OBJETIVO 2 - TREINAMENTO DO MODELO DE MACHINE LEARNING
# ==============================================================================
print("\n--- OBJETIVO 2: Treinando o Modelo de Machine Learning (versão definitiva) ---")
location_mapping = {'kyiv region': 'kyiv', 'kyivska': 'kyiv', 'kyiv and kyivska': 'kyiv', 'kyivska oblast': 'kyiv', 'kyivskyi': 'kyiv', 'kyiv city': 'kyiv', 'kharkiv region': 'kharkiv', 'kharkivska': 'kharkiv', 'odesa region': 'odesa', 'odeska': 'odesa', 'dnipropetrovsk region': 'dnipro', 'dnipropetrovska': 'dnipro', 'lviv region': 'lviv', 'lvivska': 'lviv', 'zaporizhzhia region': 'zaporizhzhia', 'mykolaiv region': 'mykolaiv', 'kherson region': 'kherson', 'poltava region': 'poltava', 'cherkasy region': 'cherkasy', 'khmelnytskyi region': 'khmelnytskyi', 'khmelnytska': 'khmelnytskyi', 'sumy region': 'sumy'}
df['main_target'] = df['target'].astype(str).str.split(',').str[0].str.strip().str.lower()
df['target_standardized'] = df['main_target'].replace(location_mapping)
model_df = df[df['launched'] > 0].copy()
model_df['interception_rate'] = model_df['destroyed'] / model_df['launched']
model_df['month'] = model_df['event_date'].dt.month
target_col = 'interception_rate'
y = model_df[target_col]
# Removendo variáveis desnecessárias ou com baixa qualidade de dados
X = model_df.drop(columns=[target_col, 'event_date', 'time_start', 'time_end', 'target', 'main_target', 'destroyed', 'carrier'])

X_train_df = X[X['year'] < 2025]
X_test_df = X[X['year'] >= 2025]
y_train = y.loc[X_train_df.index]
y_test = y.loc[X_test_df.index]

numeric_features = ['launched', 'year', 'month']
categorical_features = ['model', 'weapon_category', 'target_standardized'] # <-- Lista de features final

for col in categorical_features:
    X_train_df[col] = X_train_df[col].fillna('Unknown').astype(str)
    X_test_df[col] = X_test_df[col].fillna('Unknown').astype(str)

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder.fit(X_train_df[categorical_features])
X_train_encoded = encoder.transform(X_train_df[categorical_features])
X_test_encoded = encoder.transform(X_test_df[categorical_features])
X_train_final = np.hstack((X_train_df[numeric_features].values, X_train_encoded))
X_test_final = np.hstack((X_test_df[numeric_features].values, X_test_encoded))

if len(X_test_df) > 0 and len(X_train_df) > 0:
    model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    model.fit(X_train_final, y_train)
    predictions = model.predict(X_test_final)
    mae = mean_absolute_error(y_test, predictions)
    print(f"\nModelo Treinado. Erro Absoluto Médio (MAE) nos dados de 2025: {mae:.2%}")
else:
    model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    for col in categorical_features:
        X[col] = X[col].fillna('Unknown').astype(str)
    X_numeric_full = X[numeric_features].values
    X_encoded_full = encoder.fit_transform(X[categorical_features])
    X_final_full = np.hstack((X_numeric_full, X_encoded_full))
    model.fit(X_final_full, y)
    print("\nModelo Treinado com todos os dados disponíveis.")

joblib.dump(model, 'defense_model.joblib')
joblib.dump(encoder, 'model_encoder.joblib')
print("Modelo e Encoder salvos em arquivos .joblib.")



--- OBJETIVO 2: Treinando o Modelo de Machine Learning (versão definitiva) ---

Modelo Treinado. Erro Absoluto Médio (MAE) nos dados de 2025: 15.17%
Modelo e Encoder salvos em arquivos .joblib.


In [8]:

# ==============================================================================
# CÉLULA 6: OBJETIVO 3 - REQUISITOS PARA DEFESA (MÍSSEIS BALÍSTICOS)
# ==============================================================================
print("\n--- OBJETIVO 3: Requisitos de Defesa vs. Mísseis Balísticos (Cenário Ajustado) ---")
def calculate_required_shots(target_p0, attack_size_W, p_track, sspk):
    if sspk >= 1.0: return 1
    k_req = target_p0**(1/attack_size_W)
    if k_req >= p_track: return float('inf')
    log_term = 1 - (k_req / p_track)
    if log_term <= 0 or (1 - sspk) >= 1: return float('inf')
    n = np.log(log_term) / np.log(1 - sspk)
    return np.ceil(n)

def calculate_required_sspk(target_p0, attack_size_W, p_track, n_shots):
    k_req = target_p0**(1/attack_size_W)
    if k_req >= p_track: return float('inf')
    base_term = 1 - (k_req / p_track)
    if base_term < 0: return float('inf')
    k = 1 - (1 - base_term)**(1/n_shots)
    return k

ballistic_stats_row = stats_by_category[stats_by_category['weapon_category'] == 'ballistic missile']
if not ballistic_stats_row.empty:
    ballistic_stats = ballistic_stats_row.iloc[0]
    CURRENT_SSPK_BALLISTIC = calculate_implied_sspk(ballistic_stats['launched'], ballistic_stats['destroyed'], P_TRACK, n_shots=2)
    TARGET_P0 = 0.90
    ATTACK_SIZE_W = 10
    print(f"Analisando um objetivo de {TARGET_P0:.0%} de defesa contra {ATTACK_SIZE_W} mísseis balísticos.")
    req_n = calculate_required_shots(TARGET_P0, ATTACK_SIZE_W, P_TRACK, CURRENT_SSPK_BALLISTIC)
    print(f"\nCenário 1: Com o SSPK atual de {CURRENT_SSPK_BALLISTIC:.1%}, seriam necessários {int(req_n) if not np.isinf(req_n) else '∞'} disparos por míssil.")
    req_k = calculate_required_sspk(TARGET_P0, ATTACK_SIZE_W, P_TRACK, n_shots=2)
    print(f"Cenário 2: Com 2 disparos por míssil, o SSPK precisaria ser de {req_k:.1% if not np.isinf(req_k) else 'impossível'}.")
else:
    print("\nDados de mísseis balísticos insuficientes para análise.")



--- OBJETIVO 3: Requisitos de Defesa vs. Mísseis Balísticos (Cenário Ajustado) ---
Analisando um objetivo de 90% de defesa contra 10 mísseis balísticos.

Cenário 1: Com o SSPK atual de 9.8%, seriam necessários ∞ disparos por míssil.


ValueError: Invalid format specifier '.1% if not np.isinf(req_k) else 'impossível'' for object of type 'float'

In [ ]:

# ==============================================================================
# CÉLULA 7: OBJETIVO 3 - REQUISITOS PARA DEFESA (UAVs)
# ==============================================================================
print("\n--- OBJETIVO 3: Requisitos de Defesa vs. UAVs (Cenário Ajustado) ---")
uav_stats_row = stats_by_category[stats_by_category['weapon_category'] == 'UAV']
if not uav_stats_row.empty:
    uav_stats = uav_stats_row.iloc[0]
    CURRENT_SSPK_UAV = calculate_implied_sspk(uav_stats['launched'], uav_stats['destroyed'], P_TRACK, n_shots=1)
    TARGET_P0 = 0.90
    ATTACK_SIZE_W = 50
    print(f"Analisando um objetivo de {TARGET_P0:.0%} de defesa contra {ATTACK_SIZE_W} UAVs.")
    req_n_uav = calculate_required_shots(TARGET_P0, ATTACK_SIZE_W, P_TRACK, CURRENT_SSPK_UAV)
    print(f"\nCenário 1: Com o SSPK atual de {CURRENT_SSPK_UAV:.1%}, seria necessário um engajamento equivalente a {int(req_n_uav) if not np.isinf(req_n_uav) else '∞'} disparos por UAV.")
    req_k_uav = calculate_required_sspk(TARGET_P0, ATTACK_SIZE_W, P_TRACK, n_shots=1)
    print(f"Cenário 2: Com 1 disparo por UAV, o SSPK precisaria ser de {req_k_uav:.1% if not np.isinf(req_k_uav) else 'impossível'}.")
else:
    print("\nDados de UAVs insuficientes para análise.")


In [ ]:

# ==============================================================================
# CÉLULA 8: ANÁLISE GEOGRÁFICA COM PADRONIZAÇÃO
# ==============================================================================
print("\n--- ANÁLISE: Eficácia da Defesa por Local Alvo (Padronizado) ---")
exploded_df = df.explode('target_standardized')
if 'target_standardized' in exploded_df.columns:
    exploded_df['target_standardized'] = exploded_df['target_standardized'].str.strip().str.lower()
    exploded_df = exploded_df[exploded_df['target_standardized'] != '']
    standardized_location_stats = exploded_df.groupby('target_standardized').agg(total_launched=('launched', 'sum'), total_destroyed=('destroyed', 'sum')).reset_index()
    standardized_location_stats['interception_rate'] = np.where(standardized_location_stats['total_launched'] > 0, standardized_location_stats['total_destroyed'] / standardized_location_stats['total_launched'], 0)
    significant_targets_std = standardized_location_stats[standardized_location_stats['total_launched'] >= 20].copy()
    significant_targets_std['interception_rate_pct'] = significant_targets_std['interception_rate'].apply(lambda x: f"{x:.1%}")

    print("\nTop 10 Cidades/Regiões Mais Bem Defendidas (com 20+ ataques):")
    print(significant_targets_std.sort_values(by='interception_rate', ascending=False).head(10)[['target_standardized', 'total_launched', 'total_destroyed', 'interception_rate_pct']])

    print("\nTop 10 Cidades/Regiões Mais Vulneráveis (com 20+ ataques):")
    print(significant_targets_std.sort_values(by='interception_rate', ascending=True).head(10)[['target_standardized', 'total_launched', 'total_destroyed', 'interception_rate_pct']])
else:
    print("\nColuna 'target_standardized' não encontrada para análise geográfica.")



In [ ]:

# ==============================================================================
# CÉLULA 9: PREDIÇÃO DE ATAQUE IMINENTE (CARREGANDO MODELO SALVO)
# ==============================================================================
print("\n--- PREDIÇÃO: Executando Modelo sobre Ataque Iminente ---")
try:
    loaded_model = joblib.load('defense_model.joblib')
    loaded_encoder = joblib.load('model_encoder.joblib')
    print("Modelo e encoder carregados dos arquivos com sucesso.")


    
    imminent_attack_df = pd.read_csv('ataque_iminente.csv')
    print("Arquivo 'ataque_iminente.csv' carregado.")

    
    prediction_df = pd.merge(imminent_attack_df, weapons_df[['model', 'category']], on='model', how='left')
    prediction_df.rename(columns={'category': 'weapon_category'}, inplace=True)
    now = pd.to_datetime('now', utc=True).tz_convert('America/Sao_Paulo')
    prediction_df['year'] = now.year
    prediction_df['month'] = now.month
    prediction_df['main_target'] = prediction_df['target'].astype(str).str.strip().str.lower()
    prediction_df['target_standardized'] = prediction_df['main_target'].replace(location_mapping)
    
    categorical_features = ['model', 'weapon_category', 'target_standardized'] # <-- Lista final
    for col in categorical_features:
        prediction_df[col] = prediction_df[col].fillna('Unknown').astype(str)
    
    X_pred_categorical = loaded_encoder.transform(prediction_df[categorical_features])
    X_pred_numeric = prediction_df[['launched', 'year', 'month']].values
    X_pred_final = np.hstack((X_pred_numeric, X_pred_categorical.toarray()))
    
    imminent_predictions = loaded_model.predict(X_pred_final)
    prediction_df['predicted_interception_rate'] = imminent_predictions
    
    print("\n>>> Previsão de Resultados para o Ataque Iminente <<<")
    prediction_df['predicted_interception_rate_pct'] = prediction_df['predicted_interception_rate'].apply(lambda x: f"{x:.1%}")
    print(prediction_df[['model', 'launched', 'target', 'predicted_interception_rate_pct']])

except FileNotFoundError:
    print("\nERRO: Arquivos de modelo não encontrados. Execute a Célula 5 para treinar e salvar o modelo primeiro.")

print("\n--- FIM DO NOTEBOOK ---")